## Анализ тональности отзывов на фильмы

### В этом задании вам предлагается начать разбираться с задачей анализа тональности отзывов на примере сентимент-анализа отзывов на фильмы. Мы будем использовать стандартный датасет из nltk.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import movie_reviews

import warnings
warnings.filterwarnings('ignore')

In [2]:
def write_answer(filename, answer):
    
    with open(filename, 'w') as fout:        
        fout.write(str(answer))
        fout.close()

#### 1. Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.    

In [3]:
movie_reviews.categories()

['neg', 'pos']

In [4]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

negfeats = [[" ".join(movie_reviews.words(fileids=[f])), 0] for f in negids]
posfeats = [[" ".join(movie_reviews.words(fileids=[f])), 1] for f in posids]

features = negfeats + posfeats

In [5]:
texts = [item[0] for item in features]
target = [item[1] for item in features]

#### 2. Подсчитайте количество отзывов в выборке.   

In [6]:
write_answer('submission_1.txt', len(texts))
len(texts)

2000

#### 3. Подсчитайте долю класса 1 в выборке.   

In [7]:
write_answer('submission_2.txt', len(posfeats) / len(texts))
len(posfeats) / len(texts)

0.5

#### 4. Импортируйте CountVectorizer из <code>sklearn.feature_extraction.text</code> Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод <code>fit_transform</code> у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.    

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(texts)
write_answer('submission_3.txt', X.shape[1])

X.shape

(2000, 39659)

#### 5. Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью <code>cross_val_score</code> (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy.   

In [9]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

pipeline = make_pipeline(CountVectorizer(), LogisticRegression())

cv_score = np.mean(cross_val_score(pipeline, texts, target))
write_answer('submission_4.txt', cv_score)

cv_score

0.8424999999999999

#### 6. Аналогично accuracy, оцените качество по ROC AUC.    

In [10]:
cv_score = np.mean(cross_val_score(pipeline, texts, target, scoring='roc_auc'))
write_answer('submission_5.txt', cv_score)

cv_score

0.916455

#### 7. Обучите логистическую регрессию на всей доступной вам выборке и выведите 5 наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer.

In [11]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

clf = LogisticRegression()
clf.fit(X, target)

listed_features = dict(enumerate(vectorizer.get_feature_names()))

coeffs = clf.coef_[0]
coeffs = [np.abs(coef) for coef in coeffs]
coeffs = tuple(enumerate(coeffs))
coeffs = sorted(coeffs, key=lambda x: x[1], reverse=True)

In [12]:
top_2 = coeffs[:2]
top_2

[(2954, 0.8133386588943791), (37056, 0.6406973236580851)]

In [13]:
for index in top_2:
    print(listed_features[index[0]])

bad
unfortunately


In [14]:
write_answer('submission_6.txt', 'bad unfortunately')